In [1]:
from pubnub.pnconfiguration import PNConfiguration
from pubnub.pubnub import PubNub, SubscribeListener
from pubnub.enums import PNStatusCategory

pnconfig = PNConfiguration()
pnconfig.subscribe_key="sub-c-b1cadece-f0fa-11e3-928e-02ee2ddab7fe"
# for twitter: subkey: "sub-c-78806dd4-42a6-11e4-aed8-02ee2ddab7fe", channel: "pubnub-twitter"
# for game: subkey: "sub-c-b1cadece-f0fa-11e3-928e-02ee2ddab7fe", channel: "pubnub-game-state"
pnconfig.ssl = False

pubnub = PubNub(pnconfig)

In [2]:
def sampling(a, b, data, key):
    sample = False
    hashkey = hash(data.message[key]) % b
    if hashkey < a:
        sample = True
    return sample

my_listener = SubscribeListener()
pubnub.add_listener(my_listener)

pubnub.subscribe().channels('pubnub-game-state').execute()
my_listener.wait_for_connect()
print('connected')


print "Processing Data"
total = 100
data = []
for i in range(0, total):
    result = my_listener.wait_for_message_on('pubnub-game-state')
    if sampling(3, 10, result, "player_id"):
        data.append(result.message)
        
print len(data)

connected
Processing Data
32
